In [ ]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import math
# plt.style.use('dark_background')
plt.style.use('seaborn-notebook')
from data_load_preprocess import data
from collections import OrderedDict
from sklearn import preprocessing

import torch
from torch.autograd import Variable
from viz import viz
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")
import viz_weights

%load_ext autoreload
%autoreload 2

# load results from a directory

In [ ]:
out_dir_main = '/scratch/users/vision/yu_dl/raaz.rsk/adam_vs_sgd'
print(os.listdir(out_dir_main))
folder = 'noise'

In [ ]:
# depending on how much is saved, this may take a while
out_dir = oj(out_dir_main, folder)
fnames = sorted(os.listdir(out_dir))
results_list = [pd.Series(pkl.load(open(oj(out_dir, fname), "rb"))) for fname in fnames if not fname.startswith('weights')]
results = pd.concat(results_list, axis=1).T.infer_objects()
# results.describe()
# results.head()
# results.dtypes

# plot basic things for entire dataframe

In [ ]:
viz_weights.plot_losses(results)

In [ ]:
# ['explained_var_dicts_pca', 'explained_var_dicts_rbf', 'explained_var_dicts_lap', 'explained_var_dicts_cosine']
# act_var_dicts_test_pca (might not have pca), act_var_dicts_test_rbf
all_w = ['explained_var_dicts_pca', 'explained_var_dicts_rbf', 'explained_var_dicts_lap', 'explained_var_dicts_cosine']
acts = ['act_var_dicts_train_pca', 'act_var_dicts_test_pca', 'act_var_dicts_train_rbf', 'act_var_dicts_test_rbf']
viz_weights.plot_dims(results, xlim=None, dim_types=acts)

In [ ]:
# note these norms were squared
# calculated via np.linalg.norm(weight_dict[lay_name])**2
viz_weights.plot_weight_norms_and_margin(results)    

# look at the weights

In [ ]:
# depending on how much is saved, this may take a while
weights_list = [pd.Series(pkl.load(open(oj(out_dir, fname), "rb"))) for fname in fnames if fname.startswith('weights')]
results_weights = pd.concat(weights_list, axis=1).T.infer_objects()
# results.head()
# results_weights.dtypes

just plot them all (this takes ~2 mins)

In [ ]:
# optimizer = 'adam'
# lr = 0.1
for optimizer in ['sgd', 'adam']: # 'sgd', 'adam'
    for lr in [0.1, 0.01, 0.001]: # lr 0.1, 0.01, 0.001
        try:
            runs = results_weights[results.lr==lr]
            runs = runs[runs.optimizer==optimizer]
            run = runs.iloc[0]
            weight_dict = run.weights
            min_key, max_key = min(weight_dict.keys()), max(weight_dict.keys())
            #     print('init', optimizer, 'lr=' + str(lr))
            #     w = ws[min_key]['fc1.weight']
            #     plot_weights(w)

            print('final', optimizer, 'lr=' + str(lr))
            w = weight_dict[max_key]['fc1.weight']
            viz_weights.plot_weights(w, run.dset)
            plt.savefig('figs/' + optimizer + '_' + 'lr=' + str(lr) + '.pdf', dpi=300, bbox_inches='tight')
            # plt.show()
        except:
            pass

In [ ]:
# track weight evolution
results_weights.weights_first10

# optimizer = 'adam'
# lr = 0.1
for optimizer in ['sgd']: # 'sgd', 'adam'
    for lr in [0.1]: # lr 0.1, 0.01, 0.001
        runs = results_weights[results.lr==lr]
        runs = runs[runs.optimizer==optimizer]
        run = runs.iloc[0]
        ws = run.weights_first10
        ts = sorted(ws.keys())
        
        # select which ts to plot
        ts = ts[:10] + [10, 20, 30, 40]
        print(ts)
        
        R, C = len(ts), ws[ts[0]].shape[0]
        plt.figure(figsize=(C, R))
        for r in range(R):    
            ws_t = ws[ts[r]]
            for c in range(C):
                plt.subplot(R, C, r * C + c + 1)
                im = ws_t[c].reshape(28, 28)
                plt.imshow(im)
                
                if c == 0:                
                    plt.ylabel(ts[r])
                    plt.yticks([])
                    plt.xticks([])
                else:
                    plt.axis('off')
        plt.subplots_adjust(hspace=0, wspace=0)
        plt.savefig('figs/' + 'evol_' + optimizer + '_' + 'lr=' + str(lr) + '.pdf', dpi=300, bbox_inches='tight')        
        plt.close()

#         print(ts)
#         print()
#         print(.keys())
        '''
        min_key, max_key = min(weight_dict.keys()), max(weight_dict.keys())
        #     print('init', optimizer, 'lr=' + str(lr))
        #     w = ws[min_key]['fc1.weight']
        #     plot_weights(w)

        print('final', optimizer, 'lr=' + str(lr))
        w = weight_dict[max_key]['fc1.weight']
        viz_weights.plot_weights(w, run.dset)
        plt.savefig('figs/' + optimizer + '_' + 'lr=' + str(lr) + '.pdf', dpi=300)
        # plt.show()
        '''

*visualize pcs*

In [ ]:
# pca
optimizer, lr = 'adam', 0.01

# get w
runs = results_weights[results_weights.lr==lr]
runs = runs[runs.optimizer==optimizer]
run = runs.iloc[0]
weight_dict = run.weights
w = weight_dict[max(weight_dict.keys())]['fc1.weight']

# look at pcs
pca = PCA(n_components=w.shape[1])
pca.fit(w)
cum = deepcopy(pca.explained_variance_ratio_)
for i in range(1, len(cum)):
    cum[i] += cum[i-1]
plt.plot(cum, 'o')
plt.ylabel('cumulative explained var')
plt.xlabel('num pcs')
plt.show()

print('first pcs..')
plot_weights(pca.components_[:200], run.dset)
plt.show()

*norm distributions*

In [ ]:
# pca
plt.figure(figsize=(9, 5))
R, C = 2, 3
for j, optimizer in enumerate(['sgd', 'adam']):
    for lr in [0.1, 0.01, 0.001]:
        if not (optimizer == 'adam' and lr == 0.1):
            # get ws
            runs = results_weights[results_weights.lr==lr]
            runs = runs[runs.optimizer==optimizer]
            run = runs.iloc[0]
            weight_dict_list = run.weights
            weight_dict = weight_dict_list[max(weight_dict_list.keys())]
            ws = [weight_dict['fc1.weight'], weight_dict['fc2.weight'], weight_dict['fc3.weight']]
            w_norms = [np.sum(np.abs(w)**2, axis=1)**(1./2) for w in ws]
    #         w_norms = [np.sum(np.abs(w)**1, axis=1)**(1./1) for w in ws]        

            plt.subplot(R, C, 1 + j * C)
            plt.hist(w_norms[0], label=str(lr))
            if j == 0:
                plt.title('lay1 norms')
            plt.ylabel(optimizer)

            plt.subplot(R, C, 2 + j * C)    
            plt.hist(w_norms[1], label=str(lr))
            if j == 0:
                plt.title('lay2 norms')    

            plt.subplot(R, C, 3 + j * C)    
            plt.hist(w_norms[2], label=str(lr))
            if j == 0:
                plt.title('lay3 norms')              
plt.legend()
plt.show()